# Hyperparameter tuning Total Moodel

# Tuning a Random Forest Regression Model

Custom built expanding window validation functions will be used 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Import your helper functions
from timeseries_model_utils import (
    expanding_window_splits,
    run_expanding_cv,
    expanding_window_grid_search
)

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error


In [2]:
df = pd.read_csv("New_csv.csv")   # replace with your file
df = df.sort_values("Year").reset_index(drop=True)
df.head()


,Unnamed: 0,Year,Homicide,Assault,Sexual offences,Harm or endanger persons,"Robbery, blackmail, and extortion",Burglary,Theft,Fraud and related offences,...,Traffic and vehicle offences,Offences against justice procedures and orders,Offences against government,Environmental offences,Miscellaneous offences,Total,Population,Crime_Index,Total_Lagged,Population_lagged
0,0,1980,353,7208,1401,2176,426,11893,29103,19794,...,173948,27664,1376,3521,9651,331751,3112900,10.657297,331751.0,3112900.0
1,1,1981,366,7933,1092,2299,494,12974,29654,25007,...,142990,17981,2027,3360,9836,294409,3124900,9.421389,331751.0,3112900.0
2,2,1982,329,7760,1298,2231,504,14387,32682,26131,...,104739,13957,1203,4025,7878,252119,3156100,7.988308,294409.0,3124900.0
3,3,1983,407,8741,1449,2693,657,13851,32304,28481,...,105649,15332,736,2252,7774,254658,3199300,7.959804,252119.0,3156100.0
4,4,1984,406,9349,1682,2651,729,13905,33492,28069,...,116414,16389,1300,2235,8552,274022,3227100,8.491277,254658.0,3199300.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 24 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Unnamed: 0                                      45 non-null     int64  
 1   Year                                            45 non-null     int64  
 2   Homicide                                        45 non-null     int64  
 3   Assault                                         45 non-null     int64  
 4   Sexual offences                                 45 non-null     int64  
 5   Harm or endanger persons                        45 non-null     int64  
 6   Robbery, blackmail, and extortion               45 non-null     int64  
 7   Burglary                                        45 non-null     int64  
 8   Theft                                           45 non-null     int64  
 9   Fraud and related offences                   

In [4]:
target_col = "Total"        # or any column you want to predict
feature_cols = ["Total_Lagged","Year","Population_lagged"]

X = df[feature_cols]
y = df[target_col]


In [5]:
param_grid = {
    "n_estimators": [100, 200, 400],
    "max_depth": [3, 5, None]
}

best_params, best_score, results_df = expanding_window_grid_search(
    RandomForestRegressor,
    param_grid,
    X, y,
    initial_train_size=10,
    horizon=1,
    step=1
)

print("\nBest Parameters:", best_params)
print("Best Score:", best_score)

results_df



[GridSearch] Params {'n_estimators': 100, 'max_depth': 3} → Score = 16293.0133
[GridSearch] Params {'n_estimators': 100, 'max_depth': 5} → Score = 16273.5158
[GridSearch] Params {'n_estimators': 100, 'max_depth': None} → Score = 16251.7486
[GridSearch] Params {'n_estimators': 200, 'max_depth': 3} → Score = 15926.9663
[GridSearch] Params {'n_estimators': 200, 'max_depth': 5} → Score = 16509.3165
[GridSearch] Params {'n_estimators': 200, 'max_depth': None} → Score = 16587.4573
[GridSearch] Params {'n_estimators': 400, 'max_depth': 3} → Score = 16147.3149
[GridSearch] Params {'n_estimators': 400, 'max_depth': 5} → Score = 16095.3797
[GridSearch] Params {'n_estimators': 400, 'max_depth': None} → Score = 16321.1961

Best Parameters: {'n_estimators': np.float64(200.0), 'max_depth': np.float64(3.0)}
Best Score: 15926.966330880019


,n_estimators,max_depth,score
0,100,3.0,16293.013327
1,100,5.0,16273.515812
2,100,NaN,16251.748571
3,200,3.0,15926.966331
4,200,5.0,16509.316503
5,200,NaN,16587.457286
6,400,3.0,16147.314853
7,400,5.0,16095.379706
8,400,NaN,16321.196071


In [6]:

param_grid = {
    "alpha": [0.001, 0.01, 0.1, 1, 10, 50, 100],
    "max_iter": [1000, 5000, 10000]
}


In [7]:

X = X.ffill().bfill()


In [8]:
from sklearn.linear_model import Lasso

param_grid = {
    "alpha": [0.001, 0.01, 0.1, 1, 10, 50, 100],
    "max_iter": [1000, 5000, 10000],
}

best_params, best_score, results_df = expanding_window_grid_search(
    Lasso,
    param_grid,
    X,
    y,
    initial_train_size=10,
    horizon=1,
    step=1
)

print("\nBest Parameters (Lasso):", best_params)
print("Best Score:", best_score)
results_df


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.190e+09, tolerance: 1.473e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.906e+09, tolerance: 1.953e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.100e+09, tolerance: 2.3

[GridSearch] Params {'alpha': 0.001, 'max_iter': 1000} → Score = 20931.6176
[GridSearch] Params {'alpha': 0.001, 'max_iter': 5000} → Score = 20931.6306
[GridSearch] Params {'alpha': 0.001, 'max_iter': 10000} → Score = 20931.6306
[GridSearch] Params {'alpha': 0.01, 'max_iter': 1000} → Score = 20931.6096


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.589e+08, tolerance: 1.953e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.613e+09, tolerance: 2.376e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.551e+09, tolerance: 2.4

[GridSearch] Params {'alpha': 0.01, 'max_iter': 5000} → Score = 20931.6225
[GridSearch] Params {'alpha': 0.01, 'max_iter': 10000} → Score = 20931.6225


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.250e+09, tolerance: 2.440e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.547e+09, tolerance: 2.440e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.037e+09, tolerance: 2.4

[GridSearch] Params {'alpha': 0.1, 'max_iter': 1000} → Score = 20931.5294
[GridSearch] Params {'alpha': 0.1, 'max_iter': 5000} → Score = 20931.5419
[GridSearch] Params {'alpha': 0.1, 'max_iter': 10000} → Score = 20931.5419
[GridSearch] Params {'alpha': 1, 'max_iter': 1000} → Score = 20930.7266


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.620e+09, tolerance: 2.453e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.731e+09, tolerance: 2.734e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.449e+09, tolerance: 3.1

[GridSearch] Params {'alpha': 1, 'max_iter': 5000} → Score = 20930.7352
[GridSearch] Params {'alpha': 1, 'max_iter': 10000} → Score = 20930.7352
[GridSearch] Params {'alpha': 10, 'max_iter': 1000} → Score = 20922.6686
[GridSearch] Params {'alpha': 10, 'max_iter': 5000} → Score = 20922.6686
[GridSearch] Params {'alpha': 10, 'max_iter': 10000} → Score = 20922.6686
[GridSearch] Params {'alpha': 50, 'max_iter': 1000} → Score = 20886.8171
[GridSearch] Params {'alpha': 50, 'max_iter': 5000} → Score = 20886.8171
[GridSearch] Params {'alpha': 50, 'max_iter': 10000} → Score = 20886.8171
[GridSearch] Params {'alpha': 100, 'max_iter': 1000} → Score = 20842.0017
[GridSearch] Params {'alpha': 100, 'max_iter': 5000} → Score = 20842.0017
[GridSearch] Params {'alpha': 100, 'max_iter': 10000} → Score = 20842.0017

Best Parameters (Lasso): {'alpha': np.float64(100.0), 'max_iter': np.float64(1000.0)}
Best Score: 20842.001703861297


,alpha,max_iter,score
0,0.001,1000,20931.617567
1,0.001,5000,20931.630609
2,0.001,10000,20931.630609
3,0.010,1000,20931.609570
4,0.010,5000,20931.622542
5,0.010,10000,20931.622542
6,0.100,1000,20931.529441
7,0.100,5000,20931.541874
8,0.100,10000,20931.541874
9,1.000,1000,20930.726561


# train a lasso with alpha 0.001 and max iter 1000 and save it

In [10]:
from sklearn.linear_model import Lasso

# Initialize model
lasso = Lasso(alpha=100, max_iter=1000)

# Train model
lasso.fit(X, y)


Lasso(alpha=100)

In [11]:
import joblib

joblib.dump(lasso, "Lasso_last.joblib")

['Lasso_last.joblib']

In [12]:
lasso.predict(X[-1:])

array([215634.32445086])

In [13]:
X[-1:]

,Total_Lagged,Year,Population_lagged
44,216533.0,2024,5223100.0


# Train Random forest & Save it

In [15]:
rd = RandomForestRegressor(n_estimators=400, max_depth=None)

rd.fit(X, y)

RandomForestRegressor(n_estimators=400)

In [16]:
X[-1:]

,Total_Lagged,Year,Population_lagged
44,216533.0,2024,5223100.0


In [17]:
rd.predict(X[-1:])

array([218790.39])

In [18]:
joblib.dump(rd, "RandomForest_last.joblib")

['RandomForest_last.joblib']

# Tuning a Gradient Boosting Algorithm

In [20]:
from sklearn.ensemble import GradientBoostingRegressor

param_grid = {
    "n_estimators": [100, 300, 600],          # number of boosting stages
    "learning_rate": [0.01, 0.05, 0.1],       # shrinkage
    "max_depth": [1,2, 3, 4],                   # depth of each tree (via base estimator)
                    # only used when loss="huber" or "quantile"
}

best_params, best_score, results_df = expanding_window_grid_search(
    GradientBoostingRegressor,
    param_grid,
    X,
    y,
    initial_train_size=10,
    horizon=1,
    step=1
)

print("\nBest Parameters (GBR):", best_params)
print("Best Score:", best_score)
results_df

[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 1} → Score = 25140.8197
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 2} → Score = 19671.9939
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 3} → Score = 18326.2402
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 4} → Score = 19164.6307
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 1} → Score = 16109.9794
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 2} → Score = 13884.4394
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 3} → Score = 14909.7787
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 4} → Score = 15989.0909
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 1} → Score = 15024.2094
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 2} → S

,n_estimators,learning_rate,max_depth,score
0,100,0.01,1,25140.819700
1,100,0.01,2,19671.993895
2,100,0.01,3,18326.240233
3,100,0.01,4,19164.630656
4,100,0.05,1,16109.979404
5,100,0.05,2,13884.439403
6,100,0.05,3,14909.778652
7,100,0.05,4,15989.090950
8,100,0.10,1,15024.209401
9,100,0.10,2,13481.835612


In [21]:

import joblib


gbr = GradientBoostingRegressor(
    n_estimators=600,
    learning_rate=0.1,
    max_depth=1,
    random_state=42
)

# Train the model
gbr.fit(X, y)


GradientBoostingRegressor(max_depth=1, n_estimators=600, random_state=42)

In [55]:
joblib.dump(gbr, "total.joblib")


['total.joblib']

# Hyperparameter Tuning the Population Model

In [24]:
target_col = "Population"        # or any column you want to predict
feature_cols = ["Total_Lagged","Year","Population_lagged"]

X = df[feature_cols]
y = df[target_col]


In [25]:
from sklearn.linear_model import Lasso

param_grid = {
    "alpha": [0.001, 0.01, 0.1, 1, 10, 50, 100],
    "max_iter": [1000, 5000, 10000],
}

best_params, best_score, results_df = expanding_window_grid_search(
    Lasso,
    param_grid,
    X,
    y,
    initial_train_size=10,
    horizon=1,
    step=1
)

print("\nBest Parameters (Lasso):", best_params)
print("Best Score:", best_score)
results_df


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.744e+09, tolerance: 3.056e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.097e+09, tolerance: 3.455e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.580e+09, tolerance: 3.8

[GridSearch] Params {'alpha': 0.001, 'max_iter': 1000} → Score = 28869.4840
[GridSearch] Params {'alpha': 0.001, 'max_iter': 5000} → Score = 28869.1854
[GridSearch] Params {'alpha': 0.001, 'max_iter': 10000} → Score = 28869.1854
[GridSearch] Params {'alpha': 0.01, 'max_iter': 1000} → Score = 28869.4769


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.396e+08, tolerance: 3.056e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.090e+09, tolerance: 3.455e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.986e+09, tolerance: 3.8

[GridSearch] Params {'alpha': 0.01, 'max_iter': 5000} → Score = 28869.1784
[GridSearch] Params {'alpha': 0.01, 'max_iter': 10000} → Score = 28869.1784


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.395e+08, tolerance: 3.455e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.369e+09, tolerance: 3.871e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.439e+09, tolerance: 4.3

...

C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.796e+09, tolerance: 5.777e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.995e+09, tolerance: 6.290e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.673e+09, tolerance: 6.8

[GridSearch] Params {'alpha': 50, 'max_iter': 5000} → Score = 28830.4724
[GridSearch] Params {'alpha': 50, 'max_iter': 10000} → Score = 28830.4724


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.417e+08, tolerance: 5.777e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.183e+09, tolerance: 6.290e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.742e+09, tolerance: 6.8

[GridSearch] Params {'alpha': 100, 'max_iter': 1000} → Score = 28791.9774
[GridSearch] Params {'alpha': 100, 'max_iter': 5000} → Score = 28791.7789
[GridSearch] Params {'alpha': 100, 'max_iter': 10000} → Score = 28791.7789

Best Parameters (Lasso): {'alpha': np.float64(100.0), 'max_iter': np.float64(5000.0)}
Best Score: 28791.778854599088


,alpha,max_iter,score
0,0.001,1000,28869.484035
1,0.001,5000,28869.185399
2,0.001,10000,28869.185399
3,0.010,1000,28869.476877
4,0.010,5000,28869.178433
5,0.010,10000,28869.178433
6,0.100,1000,28869.406332
7,0.100,5000,28869.108751
8,0.100,10000,28869.108751
9,1.000,1000,28868.709635


In [26]:
gbr = GradientBoostingRegressor(
    n_estimators=600,
    learning_rate=0.1,
    max_depth=1,
    random_state=42
)

In [27]:
lasso_model = Lasso(alpha = 0.001,max_iter = 1000)

In [28]:
lasso_model.fit(X,y)

C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.573e+10, tolerance: 1.896e+09
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.001)

In [53]:
joblib.dump(lasso_model, "population.joblib")

['population.joblib']

# Recursive Model Function

In [31]:
def recursive_forecast_with_exogenous(
    X_last: pd.DataFrame,
    population_model,
    total_model,
    scaler_X_pop,
    scaler_X_total,
    population_feature_name: str,
    horizon: int = 3
):
    """
    Recursive forecast where Population_lagged is predicted first
    and then used to predict total.

    Parameters
    ----------
    X_last : pd.DataFrame
        Last known row of features (t)
    population_model : trained model
        Model that predicts Population_lagged
    total_model : trained model
        Model that predicts total
    scaler_X_pop : fitted scaler
        Scaler used for population model
    scaler_X_total : fitted scaler
        Scaler used for total model
    population_feature_name : str
        Column name of Population_lagged
    horizon : int
        Number of recursive steps

    Returns
    -------
    pd.DataFrame
        Forecasted Population and Total values
    """

    X_current = X_last.copy()
    forecasts = []

    for step in range(1, horizon + 1):

        # ---- Predict Population_lagged ----
        X_pop_scaled = scaler_X_pop.transform(X_current)
        pop_pred = population_model.predict(X_pop_scaled)[0]

        # Update population feature
        X_current[population_feature_name] = pop_pred

        # ---- Predict total ----
        X_total_scaled = scaler_X_total.transform(X_current)
        total_pred = total_model.predict(X_total_scaled)[0]

        # Store predictions
        forecasts.append({
            "step": step,
            "Population_lagged_pred": pop_pred,
            "total_pred": total_pred
        })

        # ---- Shift lags (if needed) ----
        # Example: Population_lagged_1 → Population_lagged_2
        for col in X_current.columns:
            if "_lag" in col:
                lag_num = int(col.split("_lag")[-1])
                if lag_num > 1:
                    X_current[col.replace(f"_lag{lag_num}", f"_lag{lag_num-1}")] = X_current[col]

        # Set newest lag
        X_current[population_feature_name] = pop_pred

    return pd.DataFrame(forecasts)


In [32]:
import pandas as pd

def recursive_forecast_with_exogenous(
    X_last: pd.DataFrame,
    population_model,
    total_model,
    population_feature_name: str,
    horizon: int = 3
):
    """
    Recursive forecast where Population_lagged is predicted first
    and then used to predict total.

    Parameters
    ----------
    X_last : pd.DataFrame
        Last known row of features (t)
    population_model : trained model
        Model that predicts Population_lagged
    total_model : trained model
        Model that predicts total
    population_feature_name : str
        Column name of Population_lagged
    horizon : int
        Number of recursive steps

    Returns
    -------
    pd.DataFrame
        Forecasted Population and Total values
    """

    X_current = X_last.copy()
    forecasts = []

    for step in range(1, horizon + 1):

        # ---- Predict Population_lagged ----
        pop_pred = population_model.predict(X_current)[0]

        # Update population feature
        X_current[population_feature_name] = pop_pred

        # ---- Predict total ----
        total_pred = total_model.predict(X_current)[0]

        # Store predictions
        forecasts.append({
            "step": step,
            "Population_lagged_pred": pop_pred,
            "total_pred": total_pred
        })

        # ---- Shift lagged features ----
        for col in X_current.columns:
            if "_lag" in col:
                lag_num = int(col.split("_lag")[-1])
                if lag_num > 1:
                    X_current[col.replace(
                        f"_lag{lag_num}", f"_lag{lag_num-1}"
                    )] = X_current[col]

        # Set newest lag explicitly
        X_current[population_feature_name] = pop_pred

    return pd.DataFrame(forecasts)


In [33]:
gbr.fit(X,y)

GradientBoostingRegressor(max_depth=1, n_estimators=600, random_state=42)

In [34]:
predictions = recursive_forecast_with_exogenous(X[:-1], lasso_model, gbr, "Population_lagged")

ValueError: invalid literal for int() with base 10: 'ged'

# Fix the  error

In [47]:
import pandas as pd

def recursive_forecast_with_exogenous(
    X_last: pd.DataFrame,
    population_model,
    total_model,
    population_feature_name: str,
    horizon: int = 3
):
    """
    Recursive forecast where Population_lagged is predicted first
    and then used to predict total.

    Parameters
    ----------
    X_last : pd.DataFrame
        Last known row of features (t)
    population_model : trained model
        Model that predicts Population_lagged
    total_model : trained model
        Model that predicts total
    population_feature_name : str
        Column name of Population_lagged
    horizon : int
        Number of recursive steps

    Returns
    -------
    pd.DataFrame
        Forecasted Population and Total values
    """

    X_current = X_last.copy()
    forecasts = []

    for step in range(1, horizon + 1):

        print(X_current)
        # ---- Predict Population_lagged ----
        pop_pred = population_model.predict(X_current)[0]
        print(pop_pred)
        # Update population feature
        X_current[population_feature_name] = pop_pred

        # ---- Predict total ----
        total_pred = total_model.predict(X_current)[0]

        

        # Store predictions
        forecasts.append({
            "step": step,
            "Population_lagged_pred": pop_pred,
            "total_pred": total_pred
        })

        # # ---- Shift lagged features ----
        # for col in X_current.columns:
        #     if "_lag" in col:
        #         lag_num = int(col.split("_lag")[-1])
        #         if lag_num > 1:
        #             X_current[col.replace(
        #                 f"_lag{lag_num}", f"_lag{lag_num-1}"
        #             )] = X_current[col]

        # # Set newest lag explicitly
        # X_current[population_feature_name] = pop_pred
        X_current[population_feature_name] = pop_pred
        X_current["Total_Lagged"] = total_pred
        X_current["Year"] += 1 

    return pd.DataFrame(forecasts)

In [49]:
predictions = recursive_forecast_with_exogenous(X[-1:], lasso_model, gbr, "Population_lagged")

    Total_Lagged  Year  Population_lagged
44      216533.0  2024          5223100.0
5277296.62856452
    Total_Lagged  Year  Population_lagged
44  5.266385e+06  2025       5.277297e+06
3362451.7015081774
    Total_Lagged  Year  Population_lagged
44  4.318596e+06  2026       3.362452e+06
2083267.5879166853


In [44]:
predictions

,step,Population_lagged_pred,total_pred
0,1,5.277297e+06,[5266385.188041851]
1,2,3.362452e+06,[4318596.170160055]
2,3,2.083268e+06,[4136291.8238027957]


In [51]:
X

,Total_Lagged,Year,Population_lagged
0,331751.0,1980,3112900.0
1,331751.0,1981,3112900.0
2,294409.0,1982,3124900.0
3,252119.0,1983,3156100.0
4,254658.0,1984,3199300.0
5,274022.0,1985,3227100.0
6,272657.0,1986,3247100.0
7,298086.0,1987,3246300.0
8,292500.0,1988,3274400.0
9,308817.0,1989,3283400.0
